# Traitement et stockage des données issues du scraping

Vous avez scrapé les données du site de livres et les avez stockées dans un fichier. 

L'objectif de ce notebook est de créer une base de données pour y stocker ces données.

In [1]:
import sqlite3
import pandas as pd

Lire les données du fichier sauvegardé en utilisant pandas.

In [2]:
# Lire les données du fichier que vous venez d'enregistrer
df_books = pd.read_csv('../books_infos.csv')
df_books

,Unnamed: 0,title,price,rating,availability
0,0,Frankenstein,£38.00,Two,In stock
1,1,Forever Rockers (The Rocker #12),£28.80,Three,In stock
2,2,Fighting Fate (Fighting #6),£39.24,Three,In stock
3,3,Emma,£32.93,Two,In stock
4,4,"Eat, Pray, Love",£51.32,Three,In stock
5,5,Deep Under (Walker Security #1),£47.09,Five,In stock
6,6,Choosing Our Religion: The Spiritual Lives of ...,£28.42,Four,In stock
7,7,Charlie and the Chocolate Factory (Charlie Buc...,£22.85,Three,In stock
8,8,Charity's Cross (Charles Towne Belles #4),£41.24,One,In stock
9,9,Bright Lines,£39.07,Five,In stock


## 1. Prétraitement des données

On souhaite créer la table _book_ contenant les attributs suivants : 
- id : INT, PK,
- title : TEXT,
- price : DECIMAL
- availability : BOOLEAN
- rating : INT [0:5]

Vérifier les types des colonnes du dataframe.

In [3]:
# Vérification des types de données
print(df_books.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    20 non-null     int64 
 1   title         20 non-null     object
 2   price         20 non-null     object
 3   rating        20 non-null     object
 4   availability  20 non-null     object
dtypes: int64(1), object(4)
memory usage: 932.0+ bytes
None


Dans les cellules qui suivent, des méthodes de traitement de données sont suggérées pour donner un aperçu de ce qu'il est possible de faire avec pandas.

**Il est tout à fait possible de faire autrement.**

Utiliser la méthode pandas [_astype_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html) pour convertir la colonne de titre en chaîne de caractère.

In [4]:
# Conversion de title en chaîne de caractères
df_books["title"] = df_books["title"].astype("string")

# Vérification du type de la colonne title
print(df_books["title"].dtype)

string


Pour convertir la colonne de prix en nombre décimal, il est nécessaire d'utiliser une étape intermédiaire pour retirer le caractère "£".

Il est possible par exemple d'utiliser l'attribut [.str](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.html) de la série "price".

In [5]:
# Convertir la colonne price en type décimal
df_books["price"] = df_books['price'].str.slice(1).astype("float")
# Vérification du type de la colonne price

print(df_books["price"].dtype)

float64


Convertir la colonne `availability` en boolen (True/False).

Quelles sont les valeurs possibles pour la colonne availability ?

In [6]:
# Valeurs possibles de la colonne availability


Créer une fonction qui prend en entrée la valeur de `availability` et qui renvoie True ou False en fonction de la valeur d'entrée.

In [7]:
# Fonction pour convertir la valeur de availability en booléen
def convert_availability(value : str) -> bool:
    """Convert the availability value to a boolean.

    Args:
        value (str): The availability status of the book.

    Returns:
        bool: True if the book is available, False otherwise.
    """
    

Utiliser la méthode [`apply`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html) pour appliquer la fonction à la colonne `availability`.

In [8]:
# Convertir la colonne availability en booléen (True/False)
df_books["availability"] = 

# Vérification du type de la colonne availability
print(df_books["availability"].dtype)

SyntaxError: invalid syntax (1723145458.py, line 2)

Convertir la colonne _rating_ en chiffre en utilisant un dictionnaire `rating_map` et la méthode [_map_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html).

In [ ]:
# Dictionnaire associant les notes au format initial et les valeurs numérique
ratings_map = 

df_books["rating"] =

# Vérification du type de la colonne rating
print(df_books["rating"].dtype)

: 

: 

: 

: 

In [ ]:
# Créer une fonction convert_types qui combine les traitements faits dans les cellules précédentes
def convert_types(df_books: pd.DataFrame) -> pd.DataFrame:
    """Convert the types of the DataFrame columns to appropriate types.

    Args:
        df_books (pd.DataFrame): The DataFrame containing book data.

    Returns:
        pd.DataFrame: The DataFrame with converted types.
    """




: 

: 

: 

: 

---
## 2. Insertion des données en base

Dans cette section :
- on créé une BDD sqlite  `book_store.db` (ou on se connecte à la base si elle existe déjà) en utilisant la bibliothèque python sqlite3,
- on insère les données prétraitées dans la BDD

Utiliser le [tutoriel](https://www.ionos.fr/digitalguide/sites-internet/developpement-web/sqlite3-avec-python/) pour l'utilisation de sqlite3.

Utiliser la fonction pandas adaptée qui permet d'insérer un dataframe dans une BDD.

In [ ]:
# Création de la BDD et insertion des données

: 

: 

: 

: 

Vérifier le nombre de livres présents dans la BDD en utilisant sqlite3 et la requête SQL adaptée.

In [ ]:
# Compter le nombre de livre dans la BDD

: 

: 

: 

: 